In [ ]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.prebuilt import ToolNode


tavily_tool = TavilySearchResults(max_results=2)
tools = [tavily_tool]
tool_node = ToolNode(tools=tools)

In [ ]:
from langchain_groq.chat_models import ChatGroq
llm = ChatGroq(model="gemma2-9b-it")
llm_with_tools = llm.bind_tools(tools=tools)

In [ ]:
from langgraph.graph import StateGraph,MessagesState, START,END
def chatbot_model(state:MessagesState):
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": [response]}

In [ ]:
def route(state:MessagesState):
    messages = state["messages"]
    if messages[-1].tool_calls:
        return "tools"
    else:
        return END

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

In [ ]:
workflow = StateGraph(MessagesState)

workflow.add_node("chatbot", chatbot_model)
workflow.add_node("tools",tool_node)

workflow.add_edge(START, "chatbot")
workflow.add_conditional_edges("chatbot",route,{"tools":"tools",END:END})
workflow.add_edge("tools", "chatbot")

app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "1"}}

In [ ]:
app.invoke({"messages": ["what ia capital of Bihar"]}, config, stream_mode="values")

In [ ]:
events = app.stream({"messages": ["age of narendra modi"]}, config, stream_mode="values")

In [ ]:
for event in events:
    event["messages"][-1].pretty_print()